In [1]:
# Always reload modules to have the current version
%reload_ext autoreload
%autoreload 2


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from ranking.util import json_lines as jl
import pandas as pd

In [3]:
def calculate_tfidf_for_dataset(df: pd.DataFrame) -> pd.DataFrame:
    docs = df['docContent']
    vectorizer = TfidfVectorizer()
    result = vectorizer.fit_transform(docs)
    df = pd.DataFrame.sparse.from_spmatrix(result, columns=vectorizer.get_feature_names())
    return df

def lookup_tfidf_weight(df_tfidf: pd.DataFrame, document_id, word):
    if word in df_tfidf.columns:
        return df_tfidf.at[document_id, word]
    else:
        return 0

def create_tfidf_dataset(finput):
    df = jl.read_jsonl(finput)
    tfidf = calculate_tfidf_for_dataset(df)
    df['tfidf'] = df.apply(lambda row: [(word, lookup_tfidf_weight(tfidf, row.name, word)) for word in row['docContent'].split()], axis=1)
    return df


In [4]:

df = create_tfidf_dataset('lem-all-unique-functions.jsonl')

In [20]:
at_least_five_words = df['docContent'].str.split().str.len() >= 5
filtered = df[at_least_five_words]
filtered['docQuery'] = filtered.apply(lambda row: ' '.join([word for word, weight in sorted(row['tfidf'], key=lambda word_weight: word_weight[1], reverse=True)[:5]]), axis=1)
# jl.to_jsonl(filtered, 'tf_idf.test.jsonl')

C:\Users\Paul\AppData\Local\Temp\ipykernel_18844\2941396297.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['docQuery'] = filtered.apply(lambda row: sorted(row['tfidf'], key=lambda word_weight: word_weight[1], reverse=True)[:5], axis=1)


In [15]:
filtered

,docIds,docContent,docItem,docType,tfidf,docQuery
0,[157976903],lookup function partially safe min n index,(!!!) :: [a] -> Int -> Maybe a,:: [a] -> Int -> Maybe a,"[(lookup, 0.4337269743406635), (function, 0.22...","[(lookup, 0.4337269743406635), (function, 0.22..."
1,[174280492],like selects nth element x wrap end x map,(!!!) :: [a] -> Int -> a,:: [a] -> Int -> a,"[(like, 0.2884867328649096), (selects, 0.54385...","[(like, 0.2884867328649096), (selects, 0.54385..."
2,"[104104120, 104387559, 104438376, 104513233, 1...",x n return element x nb vector element ascend ...,"(!!) :: (KnownNat n, Enum i) => Vec n a -> i -> a",":: (KnownNat n, Enum i) => Vec n a -> i -> a","[(x, 0), (n, 0), (return, 0.0692873293433096),...","[(x, 0), (n, 0), (return, 0.0692873293433096),..."
3,[1869356],list index subscript operator start index larg...,"(!!) :: (MonadThrow m, Integral n) => [a] -> n...",":: (MonadThrow m, Integral n) => [a] -> n -> m a","[(list, 0.1292507252365204), (index, 0.4972997...","[(list, 0.1292507252365204), (index, 0.4972997..."
5,"[75283085, 75284061]",index must small length list otherwise result ...,(!!) :: C n => [a] -> n -> a,:: C n => [a] -> n -> a,"[(index, 0.33472147208199016), (must, 0.313779...","[(index, 0.33472147208199016), (must, 0.313779..."
...,...,...,...,...,...,...
215648,"[74155913, 74158141]",quasiquoter denote duration microsecond see de...,µs :: QuasiQuoter,:: QuasiQuoter,"[(quasiquoter, 0.3028676587976454), (denote, 0...","[(quasiquoter, 0.3028676587976454), (denote, 0..."
215649,[58234307],aaland island aland island åland åland island,ålandIslands :: Country,:: Country,"[(aaland, 0.3046317874980479), (island, 0.6831...","[(aaland, 0.3046317874980479), (island, 0.6831..."
215651,[44388021],π pi greek small letter pi,π :: Floating α => α,:: Floating α => α,"[(π, 0), (pi, 0.7619310268922653), (greek, 0.4...","[(π, 0), (pi, 0.7619310268922653), (greek, 0.4..."
215652,[93307248],imaginary unit eisenstein integer ω sqrt 3 ι e...,ω :: EisensteinInteger,:: EisensteinInteger,"[(imaginary, 0.578018971527453), (unit, 0.3764...","[(imaginary, 0.578018971527453), (unit, 0.3764..."
